### Example - Using Monopile or GBF Installation

This tutorial uses pre compiled ORBIT configs that are stored as .yaml files in the '~/configs/ folder. There are load and save methods available in ORBIT for working with .yaml files. 

This specific notebook runs the turbine and substructure installation of a project using different intallation methods: 
- Monopile and Turbine Installation (Heavy Lift Vessel for Monopile Installation, WTIV for Turbine Installation)
- Gravity-Based Foundation Intallation (Substructure-Turbine Assembly Tow-out, no WTIV)
- Gravity-Based Foundation and Turbine Intallation (Substructure Tow-out, WTIV for Turbine Installation)

In [1]:
import os
import pandas as pd
from ORBIT import ProjectManager, load_config

weather = pd.read_csv("data/example_weather.csv", parse_dates=["datetime"])\
            .set_index("datetime")

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

### Load the project configurations

In [2]:
monopile_config = load_config("configs/example_monopile_intallation.yaml")
gbf_no_wtiv_config = load_config("configs/example_gravity-based_intallation_no_wtiv.yaml") 
gbf_with_wtiv_config = load_config("configs/example_gravity-based_intallation_with_wtiv.yaml") 

### Each Project Has Different Installation Phases

The 'TurbineInstallation' module is only required for projects involving a WTIV. The 'GravityBasedInstallation' module offers flexibility—if a WTIV is not specified in the configuration file, it models the tow-out of a fully assembled substructure and turbine; if a WTIV is present, it models the tow-out of the substructure alone.


In [3]:
print(f"Monopile and Turbine Installation (Heavy Lift Vessel for Monopile Installation, WTIV for Turbine Installation)")
print(f"Install phases: {list(monopile_config['install_phases'].keys())}\n")
print(f"Gravity-Based Foundation Intallation (Substructure-Turbine Assembly Tow-out, no WTIV)")
print(f"Install phases: {list(gbf_no_wtiv_config['install_phases'].keys())}\n")
print(f"Gravity-Based Foundation and Turbine Intallation (Substructure Tow-out, WTIV for Turbine Installation)")
print(f"Install phases: {list(gbf_with_wtiv_config['install_phases'].keys())}\n")

Monopile and Turbine Installation (Heavy Lift Vessel for Monopile Installation, WTIV for Turbine Installation)
Install phases: ['ArrayCableInstallation', 'ExportCableInstallation', 'MonopileInstallation', 'TurbineInstallation', 'OffshoreSubstationInstallation', 'ScourProtectionInstallation']

Gravity-Based Foundation Intallation (Substructure-Turbine Assembly Tow-out, no WTIV)
Install phases: ['ArrayCableInstallation', 'ExportCableInstallation', 'GravityBasedInstallation', 'OffshoreSubstationInstallation', 'ScourProtectionInstallation']

Gravity-Based Foundation and Turbine Intallation (Substructure Tow-out, WTIV for Turbine Installation)
Install phases: ['ArrayCableInstallation', 'ExportCableInstallation', 'GravityBasedInstallation', 'TurbineInstallation', 'OffshoreSubstationInstallation', 'ScourProtectionInstallation']



### Run Three Cases

This project is always being modeled with the example weather project supplied that is representative of US East Coast wind farm locations.

In [4]:
project_monopile = ProjectManager(monopile_config, weather=weather)
project_monopile.run()

ORBIT library intialized at 'C:\esteyco-no-cost-extension\ORBIT\library'


landfall dictionary will be deprecated and moved into [export_system_design][landfall].DeprecationWarning: C:\esteyco-no-cost-extension\ORBIT\ORBIT\phases\install\cable_install\export.py:84
landfall dictionary will be deprecated and moved into [export_system][landfall].

In [5]:
project_gbf_no_wtiv = ProjectManager(gbf_no_wtiv_config, weather=weather)
project_gbf_no_wtiv.run()

support_vessel will be deprecated and replaced with towing_vessels and ahts_vessel in the towing groups.
['towing_vessl_groups]['station_keeping_vessels'] will be deprecated and replaced with ['towing_vessl_groups]['ahts_vessels'].


In [6]:
project_gbf_with_wtiv = ProjectManager(gbf_with_wtiv_config, weather=weather)
project_gbf_with_wtiv.run()

### CapEx Breakdown

In [7]:
# The breakdown of project costs by module is available  at 'capex_breakdown'

df = pd.DataFrame({
    'Monopiles + WTIV': pd.Series(project_monopile.capex_breakdown),
    'GBF-Turbine Assembly Tow-out': project_gbf_no_wtiv.capex_breakdown,
    'GBF Tow-out + WTIV': pd.Series(project_gbf_with_wtiv.capex_breakdown)
}).fillna(0)

# Add Total row
df.loc['Total'] = df.sum()

# Move index to a column
df = df.reset_index().rename(columns={'index': 'CapEx Component'})

# Create a copy for display with formatting (as strings)
df_display = df.copy()

# Format numeric columns with commas (leave 'CapEx Component' alone)
for col in df_display.columns[1:]:
    df_display[col] = df_display[col].apply(lambda x: f"{x:,.0f}")

# Display the formatted version (but original df remains numeric)
df_display

,CapEx Component,Monopiles + WTIV,GBF-Turbine Assembly Tow-out,GBF Tow-out + WTIV
0,Array System,"50,455,757","50,455,757","50,455,757"
1,Array System Installation,"70,738,826","70,738,826","70,738,826"
2,Export System,"1,432,941,157","1,432,941,157","1,432,941,157"
3,Export System Installation,"92,268,096","92,268,096","92,268,096"
4,Offshore Substation,"307,307,330","307,307,330","307,307,330"
5,Offshore Substation Installation,"5,378,378","5,378,378","5,378,378"
6,Project,"185,582,083","185,582,083","185,582,083"
7,Scour Protection,"5,294,000","5,294,000","5,294,000"
8,Scour Protection Installation,"19,419,467","19,419,467","19,419,467"
9,Soft,"811,054,849","776,361,865","790,699,405"


### Substructure and Turbine Installation CapEx Breakdown

In [8]:
# Filter only the rows of interest
subset_df = df[df['CapEx Component'].isin(['Substructure Installation', 'Turbine Installation'])].copy()

# Add Total row (sum of these two rows)
total_row = {
    'CapEx Component': 'Total',
}
for col in subset_df.columns[1:]:
    total_row[col] = subset_df[col].sum()
subset_df = pd.concat([subset_df, pd.DataFrame([total_row])], ignore_index=True)

# Optional: format for display (with commas)
subset_display = subset_df.copy()
for col in subset_display.columns[1:]:
    subset_display[col] = subset_display[col].apply(lambda x: f"{x:,.0f}")

# Show formatted version
subset_display

,CapEx Component,Monopiles + WTIV,GBF-Turbine Assembly Tow-out,GBF Tow-out + WTIV
0,Substructure Installation,"60,062,259","127,355,430","41,845,350"
1,Turbine Installation,"107,561,827",0,"107,561,827"
2,Total,"167,624,086","127,355,430","149,407,178"


### Installation Actions

Save the installation sequences from each configuration file on a csv (just for substructure and turbine), and show total durations by installation step.

In [9]:
df = pd.DataFrame(project_monopile.actions)  
project_monopile_turbine_install = df.loc[df['phase']=="TurbineInstallation"]
project_monopile_turbine_install.to_csv("monopile_case_turbine_install.csv")
project_monopile_monopile_install = df.loc[df['phase']=="MonopileInstallation"]
project_monopile_monopile_install.to_csv("monopile_case_monopile_install.csv")

project_monopile_monopile_install.groupby(["action"]).sum()['duration']

action
Bolt TP                     200.000000
Crane Reequip               100.000000
Delay                       545.000000
Drive Monopile               75.000000
Fasten Monopile             600.000000
Fasten Transition Piece     400.000000
Jackdown                     21.666667
Jackup                       21.666667
Lower Monopile                0.250000
Lower TP                     50.000000
Mobilize                    168.000000
Position Onsite             100.000000
Release Monopile            150.000000
Release Transition Piece    100.000000
RovSurvey                    50.000000
Transit                     396.000000
Upend Monopile               40.816890
Name: duration, dtype: float64

In [10]:
df = pd.DataFrame(project_gbf_no_wtiv.actions)  
project_gbf_no_wtiv_turbine_install = df.loc[df['phase']=="TurbineInstallation"]
project_gbf_no_wtiv_turbine_install.to_csv("gbf_no_wtiv_case_turbine_install.csv")
project_gbf_no_wtiv_gbf_install = df.loc[df['phase']=="GravityBasedInstallation"]
project_gbf_no_wtiv_gbf_install.to_csv("gbf_no_wtiv_case_gbf_install.csv")

project_gbf_no_wtiv_gbf_install.groupby(["action"]).sum()['duration']

action
Delay                                                4494.000000
Delay: No Completed Assemblies Available             6391.428571
Delay: No Substructure Storage Available             9488.000000
Delay: Not enough vessels for gravity foundations    7474.928571
Electrical Completion                                3600.000000
Grout GBF                                             300.000000
Lift and Attach Blade                                 525.000000
Lift and Attach Nacelle                               600.000000
Lift and Attach Tower Section                         400.000000
Mechanical Completion                                1200.000000
Mobilize                                               72.000000
Move Substructure                                     400.000000
Position Substructure                                 250.000000
Positioning Support                                  3209.571429
Prepare for Turbine Assembly                          600.000000
Pump Ballast      

In [11]:
df = pd.DataFrame(project_gbf_with_wtiv.actions)  
project_gbf_with_wtiv_turbine_install = df.loc[df['phase']=="TurbineInstallation"]
project_gbf_with_wtiv_turbine_install.to_csv("gbf_with_wtiv_case_turbine_install.csv")
project_gbf_with_wtiv_gbf_install = df.loc[df['phase']=="GravityBasedInstallation"]
project_gbf_with_wtiv_gbf_install.to_csv("gbf_with_wtiv_case_gbf_install.csv")

project_gbf_with_wtiv_gbf_install.groupby(["action"]).sum()['duration']

action
Delay                                                1310.000000
Delay: No Substructure Storage Available             3650.429000
Delay: Not enough vessels for gravity foundations    1400.000000
Grout GBF                                             300.000000
Mobilize                                               72.000000
Position Substructure                                 250.000000
Positioning Support                                  2518.571429
Pump Ballast                                          600.000000
ROV Survey                                             50.000000
Substructure Assembly                                   0.000000
Tow Substructure                                     1000.000000
Transit                                               445.714286
Name: duration, dtype: float64